# Movie Success Analysis

Author: Lerato Matlala

## Part 1

### Data Loading

***Import Libraries***

In [3]:
#Import libraries
import pandas as pd
import numpy as np
import os
from sqlalchemy.types import *

from sqlalchemy import create_engine
from sqlalchemy_utils import create_database, database_exists
import sqlite3

***Load Data***

In [4]:
# Read in title Basics.tsv
basics_url ="https://datasets.imdbws.com/title.basics.tsv.gz"

title_basic_df = pd.read_csv(basics_url, sep='\t', low_memory=False)
title_basic_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [5]:
# Read in title.akas.tsv
title_akas_url = "https://datasets.imdbws.com/title.akas.tsv.gz"
title_akas_df = pd.read_csv(title_akas_url, sep='\t', low_memory=False)
title_akas_df.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [6]:
# Read in title.akas.tsv
ratings_url = "https://datasets.imdbws.com/title.ratings.tsv.gz"
title_ratings_df = pd.read_csv(ratings_url, sep='\t', low_memory=False)
title_ratings_df.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2007
1,tt0000002,5.8,269
2,tt0000003,6.5,1911
3,tt0000004,5.5,178
4,tt0000005,6.2,2692


### Data Cleaning

#### Handling \N Placeholder Values

In [7]:
#Count null values in title basic before handling \N placeholder values
title_basic_df.isnull().sum()

tconst             0
titleType          0
primaryTitle      11
originalTitle     11
isAdult            0
startYear          0
endYear            0
runtimeMinutes     0
genres            18
dtype: int64

In [8]:
#Count null values in title akas before handling \N placeholder values
title_akas_df.isnull().sum()

titleId              0
ordering             0
title                5
region             117
language             0
types                0
attributes           0
isOriginalTitle      0
dtype: int64

In [9]:
#Count null values in title ratings before handling \N placeholder values
title_ratings_df.isnull().sum()

tconst           0
averageRating    0
numVotes         0
dtype: int64

In [10]:
# Replace '\N' with np.nan for each DataFrame 
title_basic_df.replace({'\\N': np.nan}, inplace=True)
title_akas_df.replace({'\\N': np.nan}, inplace=True)
title_ratings_df.replace({'\\N': np.nan}, inplace=True)

In [11]:
#Count null values in title basic after handling \N placeholder values
title_basic_df.isnull().sum()

tconst                   0
titleType                0
primaryTitle            11
originalTitle           11
isAdult                  1
startYear          1382004
endYear           10199614
runtimeMinutes     7207667
genres              460268
dtype: int64

In [12]:
#Count null values in title akas after handling \N placeholder values
title_akas_df.isnull().sum()

titleId                   0
ordering                  0
title                     5
region              1908160
language            6806931
types              32051544
attributes         37499124
isOriginalTitle        2077
dtype: int64

In [13]:
#Count null values in title ratings after handling \N placeholder values
title_ratings_df.isnull().sum()

tconst           0
averageRating    0
numVotes         0
dtype: int64

#### Filter Out Movies

In [14]:
# title_basic_df Dataframe shape before filtering out movies
title_basic_df.shape

(10314337, 9)

In [15]:
#title_akas_df Dataframe shape before filtering out movies
title_akas_df.shape

(37768311, 8)

In [16]:
# title_ratings_df Dataframe shape before filtering out movies
title_ratings_df.shape

(1368573, 3)

-  **Exclude any movie with missing values for genre or runtime**

In [17]:
# Remove rows where 'genres' or 'runtimeMinutes' is missing
filtered_title_basic_df = title_basic_df.dropna(subset=['genres', 'runtimeMinutes'])

# Collect the IDs of the filtered rows
filtered_tconst = filtered_title_basic_df['tconst'].tolist()

# Filter corresponding rows in title_akas_df and title_ratings_df based on the collected IDs
filtered_title_akas_df = title_akas_df[title_akas_df['titleId'].isin(filtered_tconst)]
filtered_title_ratings_df = title_ratings_df[title_ratings_df['tconst'].isin(filtered_tconst)]

In [18]:
# title_basic_df Dataframe shape after filtering out movies with missing values for genre or runtime
filtered_title_basic_df.shape

(3024463, 9)

In [19]:
# title_akas_df Dataframe shape after filtering out movies with missing values for genre or runtime
filtered_title_akas_df.shape

(10192383, 8)

In [20]:
#title_ratings_df Dataframe shape after filtering out movies with missing values for genre or runtime
filtered_title_ratings_df.shape

(961733, 3)

- **Include only full-length movies (titleType = "movie")**

In [21]:
# Include only rows where 'titleType' is 'movie'
filtered_title_basic_df = filtered_title_basic_df[filtered_title_basic_df['titleType'] == 'movie']

# Collect the IDs of the filtered rows
filtered_tconst = filtered_title_basic_df['tconst'].tolist()

# Filter corresponding rows in title_akas_df and title_ratings_df based on the collected IDs
filtered_title_akas_df = title_akas_df[title_akas_df['titleId'].isin(filtered_tconst)]
filtered_title_ratings_df = title_ratings_df[title_ratings_df['tconst'].isin(filtered_tconst)]


In [22]:
# filtered_title_basic_df Dataframe shape after including only rows where 'titleType' is 'movie'
filtered_title_basic_df.shape

(392280, 9)

In [23]:
#filtered_title_akas_df Dataframe shape after including only rows where 'titleType' is 'movie'
filtered_title_akas_df.shape

(2461799, 8)

In [24]:
#filtered_title_ratings_df Dataframe shape after including only rows where 'titleType' is 'movie'
filtered_title_ratings_df.shape

(264414, 3)

- **Include only fictional movies (not from documentary genre)**

In [25]:
# Include only rows where 'genres' does not contain 'Documentary'
filtered_title_basic_df = filtered_title_basic_df[(~filtered_title_basic_df['genres'].str.contains('Documentary',case=False))]

# Collect the IDs of the filtered rows
filtered_tconst = filtered_title_basic_df['tconst'].tolist()

# Filter corresponding rows in title_akas_df and title_ratings_df based on the collected IDs
filtered_title_akas_df = title_akas_df[title_akas_df['titleId'].isin(filtered_tconst)]
filtered_title_ratings_df = title_ratings_df[title_ratings_df['tconst'].isin(filtered_tconst)]


In [26]:
#filtered_title_basic_df Dataframe shape after including only rows where 'genres' does not contain 'Documentary'
filtered_title_basic_df.shape

(296857, 9)

In [27]:
#filtered_title_akas_df Dataframe shape after including only rows where 'genres' does not contain 'Documentary'
filtered_title_akas_df.shape

(2167552, 8)

In [28]:
#filtered_title_ratings_df Dataframe shape after including only rows where 'genres' does not contain 'Documentary'
filtered_title_ratings_df.shape

(221164, 3)

- **Include only movies that were released 2000 - 2021 (include 2000 and 2021)**

In [29]:
# Convert 'startYear' to numeric (integers) if it's in object format
filtered_title_basic_df['startYear'] = pd.to_numeric(filtered_title_basic_df['startYear'], errors='coerce')

# Filter 'startYear' values between 2000 and 2022
filtered_title_basic_df = filtered_title_basic_df[(filtered_title_basic_df['startYear'] >= 2000) & (filtered_title_basic_df['startYear'] <= 2022)]


# Include only rows where 'startYear' is either 2000 or 2001
#filtered_title_basic_df = filtered_title_basic_df[((filtered_title_basic_df['startYear'] == '2000') | (filtered_title_basic_df['startYear'] == '2001'))].copy()

# Collect the IDs of the filtered rows
filtered_tconst = filtered_title_basic_df['tconst'].tolist()

# Filter corresponding rows in title_akas_df and title_ratings_df based on the collected IDs
filtered_title_akas_df = title_akas_df[title_akas_df['titleId'].isin(filtered_tconst)]
filtered_title_ratings_df = title_ratings_df[title_ratings_df['tconst'].isin(filtered_tconst)]

In [30]:
#filtered_title_basic_df Dataframe shape after including only rows where 'startYear' is between 2000 and 2022
filtered_title_basic_df.shape

(148775, 9)

In [31]:
#filtered_title_akas_df Dataframe shape after including only rows where 'startYear' is between 2000 and 2022
filtered_title_akas_df.shape

(1025400, 8)

In [32]:
#filtered_title_ratings_df Dataframe shape after including only rows where 'startYear' is between 2000 and 2022
filtered_title_ratings_df.shape

(111942, 3)

- **Include only movies that were released in the United States**

In [33]:
# Include only rows where 'country' is 'USA'
filtered_title_akas_df = filtered_title_akas_df[filtered_title_akas_df['region'] == 'US']

# Collect the IDs of the filtered rows
filtered_titleId = filtered_title_akas_df['titleId'].tolist()

# Filter corresponding rows in filtered_title_basic_df and filtered_title_ratings_df based on the collected IDs
filtered_title_basic_df = filtered_title_basic_df[filtered_title_basic_df['tconst'].isin(filtered_titleId)]
filtered_title_ratings_df = filtered_title_ratings_df[filtered_title_ratings_df['tconst'].isin(filtered_titleId)]


In [34]:
#filtered_title_basic_df Dataframe shape after filtering out movies that were released in the US
filtered_title_basic_df.shape

(87493, 9)

In [35]:
#filtered_title_akas_df Dataframe shape after filtering out movies that were released in the US
filtered_title_akas_df.shape

(97315, 8)

In [36]:
#filtered_title_ratings_df Dataframe shape after filtering out movies that were released in the US
filtered_title_ratings_df.shape

(72584, 3)

#### Dataframes Summaries

**title_basic_df**

In [37]:
filtered_title_basic_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87493 entries, 34800 to 10314103
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          87493 non-null  object 
 1   titleType       87493 non-null  object 
 2   primaryTitle    87493 non-null  object 
 3   originalTitle   87493 non-null  object 
 4   isAdult         87493 non-null  object 
 5   startYear       87493 non-null  float64
 6   endYear         0 non-null      object 
 7   runtimeMinutes  87493 non-null  object 
 8   genres          87493 non-null  object 
dtypes: float64(1), object(8)
memory usage: 6.7+ MB


**title_akas_df**

In [38]:
filtered_title_akas_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97315 entries, 203969 to 37767570
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   titleId          97315 non-null  object
 1   ordering         97315 non-null  int64 
 2   title            97315 non-null  object
 3   region           97315 non-null  object
 4   language         962 non-null    object
 5   types            89568 non-null  object
 6   attributes       4369 non-null   object
 7   isOriginalTitle  97315 non-null  object
dtypes: int64(1), object(7)
memory usage: 6.7+ MB


**title_ratings_df**

In [39]:
filtered_title_ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72584 entries, 17967 to 1368544
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tconst         72584 non-null  object 
 1   averageRating  72584 non-null  float64
 2   numVotes       72584 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 2.2+ MB


#### Save Dataframes

In [40]:
#Create Data folder
os.makedirs('Data/',exist_ok=True) 

# Confirm folder created
os.listdir("Data/")

['filtered_title_akas.csv.gz',
 'combined_tmdb_data.csv.gz',
 'filtered_title_ratings.csv.gz',
 '.ipynb_checkpoints',
 'filtered_title_basic.csv.gz']

In [41]:
# Save filtered_title_basic_df DataFrame to a compressed CSV file in the "Data/" folder
filtered_title_basic_df.to_csv('Data/filtered_title_basic.csv.gz', index=False, compression='gzip')

In [42]:
# Save filtered_title_akas_df DataFrame to a compressed CSV file in the "Data/" folder
filtered_title_akas_df.to_csv('Data/filtered_title_akas.csv.gz', index=False, compression='gzip')

In [43]:
# Save filtered_title_ratings_df DataFrame to a compressed CSV file in the "Data/" folder
filtered_title_ratings_df.to_csv('Data/filtered_title_ratings.csv.gz', index=False, compression='gzip')

## Part 2: Extract from TMDB

### Import Libraries

In [44]:
#Import Libraries
import os, time,json
import tmdbsimple as tmdb 
from tqdm.notebook import tqdm_notebook
import pandas as pd
import glob
import locale

In [45]:
#Loading TMDB API Key from JSON File and Setting it for TMDb API Access
with open('/Users/leratomatlala/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

tmdb.API_KEY =  login['api-key']

In [46]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)


### TMDB API Calls

In [47]:
#Function to Retrieve Movie Information with Certification Rating for a Given Movie ID
def get_movie_with_rating(movie_id):
    """Adapted from source = https://github.com/celiao/tmdbsimple"""
    movie  = tmdb.Movies(movie_id)
    info = movie.info()
    releases = movie.releases()
    for c in releases['countries']:
        if c['iso_3166_1'] == 'US':
            info['certification'] = c['certification']
            
            return info

In [51]:
# Load in the dataframe from project part 1 as basics:
basics_url ="Data/filtered_title_basic.csv.gz"
basics = pd.read_csv(basics_url,  low_memory=False)

# Create Required Lists for the Loop
#YEARS_TO_GET = [2000, 2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021]
YEARS_TO_GET = [2000, 2001]
errors = []

# Start OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    
    # Define the folder to store results
    FOLDER = "Data/"
    os.makedirs(FOLDER, exist_ok=True)
    os.listdir(FOLDER)

    # Define the JSON file to store results for the year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    
    # Check if the file exists
    file_exists = os.path.isfile(JSON_FILE)
    
    # If it does not exist: create it
    if not file_exists:
        with open(JSON_FILE, 'w') as f:
            json.dump([{'imdb_id': 0}], f)
    
    # Define/filter the IDs to call
    df = basics.loc[basics['startYear'] == YEAR].copy()
    movie_ids = df['tconst'].copy()
    
    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    
    # Check for and filter out movie IDs that already exist
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    
    # Start INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                              desc=f'Movies from {YEAR}',
                              position=1,
                              leave=True):
        try:
            temp = get_movie_with_rating(movie_id)
            temp_filtered = {
                    'imdb_id' : temp.get('imdb_id'),
                    'revenue': temp.get('revenue'),
                    'budget': temp.get('budget'),
                    'certification': temp.get('certification')
            }
            write_json(temp_filtered, JSON_FILE)
            time.sleep(0.02)
        except Exception as e:
            errors.append([movie_id, e])
     # After the Inner Loop
# Read the JSON file into a dataframe
    print(f"Contents of {JSON_FILE}:")
    with open(JSON_FILE, 'r') as f:
        print(f.read())

    #final_year_df = pd.read_json(JSON_FILE)
       

    # After the Inner Loop
    # Read the JSON file into a dataframe
    final_year_df = pd.read_json(JSON_FILE)
    
    # Save the year's results as csv.gz file
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)


YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1468 [00:00<?, ?it/s]

Contents of Data/tmdb_api_results_2000.json:
[{"imdb_id": 0}, {"imdb_id": "tt0113026", "revenue": 0, "budget": 10000000, "certification": ""}, {"imdb_id": "tt0113092", "revenue": 0, "budget": 0, "certification": ""}, {"imdb_id": "tt0118694", "revenue": 14204632, "budget": 150000, "certification": "PG"}, {"imdb_id": "tt0118852", "revenue": 0, "budget": 0, "certification": "R"}, {"imdb_id": "tt0119273", "revenue": 0, "budget": 15000000, "certification": "R"}, {"imdb_id": "tt0119495", "revenue": 0, "budget": 0, "certification": "R"}, {"imdb_id": "tt0120202", "revenue": 9206279, "budget": 0, "certification": ""}, {"imdb_id": "tt0120263", "revenue": 80334, "budget": 0, "certification": ""}, {"imdb_id": "tt0120467", "revenue": 14904, "budget": 120000, "certification": "R"}, {"imdb_id": "tt0120630", "revenue": 224834564, "budget": 45000000, "certification": "G"}, {"imdb_id": "tt0120698", "revenue": 0, "budget": 0, "certification": ""}, {"imdb_id": "tt0120733", "revenue": 0, "budget": 0, "cert

Movies from 2001:   0%|          | 0/1584 [00:00<?, ?it/s]

Contents of Data/tmdb_api_results_2001.json:
[{"imdb_id": 0}, {"imdb_id": "tt0035423", "revenue": 76019048, "budget": 48000000, "certification": "PG-13"}, {"imdb_id": "tt0118589", "revenue": 5271666, "budget": 22000000, "certification": "PG-13"}, {"imdb_id": "tt0118652", "revenue": 0, "budget": 1000000, "certification": "R"}, {"imdb_id": "tt0120166", "revenue": 0, "budget": 0, "certification": "NR"}, {"imdb_id": "tt0120681", "revenue": 74558115, "budget": 35000000, "certification": "R"}, {"imdb_id": "tt0120737", "revenue": 871368364, "budget": 93000000, "certification": "PG-13"}, {"imdb_id": "tt0120807", "revenue": 0, "budget": 0, "certification": ""}, {"imdb_id": "tt0120824", "revenue": 24690441, "budget": 38000000, "certification": "R"}, {"imdb_id": "tt0123581", "revenue": 0, "budget": 0, "certification": ""}, {"imdb_id": "tt0124889", "revenue": 0, "budget": 0, "certification": ""}, {"imdb_id": "tt0125022", "revenue": 57756408, "budget": 35000000, "certification": "PG-13"}, {"imdb_id

### Combine Data

In [52]:
# Define the folder where CSV files are stored
folder = "Data/"

# Find all CSV files in the folder
csv_files = glob.glob(f"{folder}final_tmdb_data_*.csv.gz")

# Initialize an empty list to store dataframes
dfs = []

# Load and concatenate each CSV file
for csv_file in csv_files:
    df = pd.read_csv(csv_file, compression="gzip")
    dfs.append(df)

# Concatenate all dataframes into one
combined_df = pd.concat(dfs, ignore_index=True)

# Display the combined dataframe
combined_df.head()

,imdb_id,revenue,budget,certification
0,0,NaN,NaN,NaN
1,tt0113026,0.0,10000000.0,NaN
2,tt0113092,0.0,0.0,NaN
3,tt0118694,14204632.0,150000.0,PG
4,tt0118852,0.0,0.0,R


***Save Combined Data***

In [53]:
combined_df.to_csv(f"{folder}combined_tmdb_data.csv.gz", compression="gzip", index=False)

***Exploratory Data Analysis***

In [54]:
# Task 1: Count movies with valid financial information
valid_financial_info_movies = combined_df[(combined_df['budget'] > 0) | (combined_df['revenue'] > 0)]
num_movies_with_valid_financial_info = len(valid_financial_info_movies)
print(f"Number of movies with valid financial information: {num_movies_with_valid_financial_info}")

Number of movies with valid financial information: 569


In [55]:
# Task 2: Count movies in each certification category
certification_counts = combined_df['certification'].value_counts()
print("Number of movies in each certification category:")
print(certification_counts)

Number of movies in each certification category:
R          449
PG-13      173
NR          68
PG          60
G           24
NC-17        5
Unrated      1
Name: certification, dtype: int64


In [56]:
# Task 3: Average revenue per certification category (excluding 0 values)
average_revenue_per_certification = valid_financial_info_movies.groupby('certification')['revenue'].mean()
locale.setlocale(locale.LC_NUMERIC, 'en_US.UTF-8')

# Format the numbers for better readability
average_revenue_per_certification_formatted = average_revenue_per_certification.apply(lambda x: locale.format_string("%d", x, grouping=True))

# Print the formatted results
print("Average revenue per certification category:")
print(average_revenue_per_certification_formatted)

Average revenue per certification category:
certification
G        110,957,400
NR         5,311,700
PG        98,197,389
PG-13     92,298,695
R         31,755,247
Name: revenue, dtype: object


In [57]:
# Task 4: Average budget per certification category (excluding 0 values)
average_budget_per_certification = valid_financial_info_movies.groupby('certification')['budget'].mean()
average_budget_per_certification_formatted = average_budget_per_certification.apply(lambda x: locale.format_string("%d", x, grouping=True))

print("Average budget per certification category:")
print(average_budget_per_certification_formatted)

Average budget per certification category:
certification
G        35,750,000
NR        2,259,649
PG       40,514,206
PG-13    42,407,788
R        19,060,474
Name: budget, dtype: object


## Part 3

### Import Data

In [58]:
# Read in title Basics.tsv
basics ="Data/filtered_title_basic.csv.gz"

filtered_title_basic_df = pd.read_csv(basics, low_memory=False)
filtered_title_basic_df_copy = pd.read_csv(basics, low_memory=False)
filtered_title_basic_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0082328,movie,Embodiment of Evil,Encarnação do Demônio,0,2008.0,NaN,94,Horror
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"


In [59]:
# Read in title Basics.tsv
basics2 ="Data/filtered_title_ratings.csv.gz"

filtered_title_ratings_df = pd.read_csv(basics2, low_memory=False)
filtered_title_ratings_df.head()

,tconst,averageRating,numVotes
0,tt0035423,6.4,87937
1,tt0062336,6.4,183
2,tt0069049,6.7,7873
3,tt0082328,5.9,1758
4,tt0088751,5.3,344


### Getting a List of Unique Genres

In [60]:
## create a col with a list of genres
filtered_title_basic_df['genres_split'] = filtered_title_basic_df['genres'].str.split(',')
filtered_title_basic_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,genres_split
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance","[Comedy, Fantasy, Romance]"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama,[Drama]
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama,[Drama]
3,tt0082328,movie,Embodiment of Evil,Encarnação do Demônio,0,2008.0,NaN,94,Horror,[Horror]
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi","[Comedy, Horror, Sci-Fi]"


In [61]:
#separate the list of genres into new rows: one row for each genre a movie belonged to.
exploded_genres = filtered_title_basic_df.explode('genres_split')
exploded_genres.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,genres_split
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance",Comedy
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance",Fantasy
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance",Romance
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama,Drama


In [62]:
#get the unique genres from the genres_split colum
unique_genres = sorted(exploded_genres['genres_split'].unique())
unique_genres

['Action',
 'Adult',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Drama',
 'Family',
 'Fantasy',
 'Game-Show',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Sport',
 'Talk-Show',
 'Thriller',
 'War',
 'Western']

### Create a new title_genres table

In [63]:
#create a new title_genres from the exploded df.
title_genres = exploded_genres[['tconst', 'genres_split']].copy()
title_genres.head()

,tconst,genres_split
0,tt0035423,Comedy
0,tt0035423,Fantasy
0,tt0035423,Romance
1,tt0062336,Drama
2,tt0069049,Drama


### Create a genre mapper dictionary to replace string genres with integers

In [64]:
## Making the genre mapper dictionary
genre_ints = range(len(unique_genres))
genre_map = dict(zip(unique_genres, genre_ints))
genre_map

{'Action': 0,
 'Adult': 1,
 'Adventure': 2,
 'Animation': 3,
 'Biography': 4,
 'Comedy': 5,
 'Crime': 6,
 'Drama': 7,
 'Family': 8,
 'Fantasy': 9,
 'Game-Show': 10,
 'History': 11,
 'Horror': 12,
 'Music': 13,
 'Musical': 14,
 'Mystery': 15,
 'News': 16,
 'Reality-TV': 17,
 'Romance': 18,
 'Sci-Fi': 19,
 'Sport': 20,
 'Talk-Show': 21,
 'Thriller': 22,
 'War': 23,
 'Western': 24}

### Replace the string genres in title_genres with the new integer ids.

In [65]:
## make new integer genre_id and drop string genres
title_genres['genre_id'] = title_genres['genres_split'].map(genre_map)
title_genres = title_genres.drop(columns='genres_split')
title_genres.head()

,tconst,genre_id
0,tt0035423,5
0,tt0035423,9
0,tt0035423,18
1,tt0062336,7
2,tt0069049,7


### Convert the genre map dictionary into a dataframe.

In [66]:
## make new integer genre_id and drop string genres
genres = pd.DataFrame({'Genre_Name':genre_map.keys(),
                             'Genre_ID':genre_map.values()})
genres.head()

,Genre_Name,Genre_ID
0,Action,0
1,Adult,1
2,Adventure,2
3,Animation,3
4,Biography,4


### Saving the MySQL tables with tconst as the primary key

#### Saving title_basic

In [67]:
## Calculate max string lengths for object columns
key_len = filtered_title_basic_df['tconst'].fillna('').map(len).max()
title_len = filtered_title_basic_df['primaryTitle'].fillna('').map(len).max()

## Create a schema dictonary using Sqlalchemy datatype objects
title_basic_schema = {
    "tconst": String(key_len+1), 
    "primaryTitle": Text(title_len+1),
    'startYear':Float(),
    'endYear':Float(),
    'runtimeMinutes':Integer()}

In [68]:
# Save to sql with dtype and index=False
filtered_title_basic_df_copy.to_sql('title_basics',engine,dtype=title_basic_schema,if_exists='replace',index=False)

NameError: name 'engine' is not defined

#### Saving title_ratings

In [ ]:
# Create a schema dictonary using Sqlalchemy datatype objects
title_rating_schema = {
    "tconst": String(key_len+1), 
    'averageRating':Float(),
    'numVotes':Integer()}

filtered_title_ratings_df.to_sql('title_ratings',engine,dtype=title_rating_schema,if_exists='replace',index=False)

#### Saving title_genres

In [ ]:
title_genres.to_sql('title_genres',engine,if_exists='replace',index=False)


#### Saving genres

In [ ]:
## Set the dataframe index and use index=True 
genres.set_index('Genre_ID').to_sql('genres',engine,index=True)

### Query the Database tables

***querying title basics***

In [ ]:
title_basics_query = """SELECT *
            FROM Movies.title_basics
            LIMIT 5;
        """
pd.read_sql(title_basics_query, engine)


***querying title ratings***

In [ ]:
title_ratings_query = """SELECT *
            FROM Movies.title_ratings
            LIMIT 5;
        """
pd.read_sql(title_ratings_query, engine)


***querying genres***

In [ ]:
genre_query = """SELECT *
            FROM Movies.genres
            LIMIT 5;
        """
pd.read_sql(genre_query, engine)


***querying title genres***

In [ ]:
title_genres_query = """SELECT *
            FROM Movies.title_genres
            LIMIT 5;
        """
pd.read_sql(title_genres_query, engine)
